## Evaluating bart base in validation dataset

In [3]:
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration
from rouge_score import rouge_scorer
from torch import optim

# Load the data from the CSV file
train_data = pd.read_csv("data/train.csv")
val_data = pd.read_csv("data/validation.csv")
# Select the columns containing the texts and summaries
train_texts = train_data["text"].tolist()
train_summaries = train_data["titles"].tolist()

val_texts = val_data["text"].tolist()
val_summaries = val_data["titles"].tolist()

# Load the tokenizer and the Bart model for text generation
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")


In [4]:
# Charger le tokenizer et le modèle BART pour la génération de texte
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("trained_model")
import torch
device = "cuda"
# Evaluation loop
all_scores = []

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
for i in range(len(val_texts)):
    input_text = val_texts[i]
    input_encoding = tokenizer(input_text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = input_encoding.input_ids.to(model.device)
    
    # Generate summary
    output_ids = model.generate(input_ids)
    output_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    # Compute ROUGE score
    rouge_scores = scorer.score(output_summary, val_summaries[i])
    rouge_l_fmeasure = rouge_scores['rougeL'].fmeasure
    
    print(f"Example {i+1}, ROUGE-L F-Score: {rouge_l_fmeasure}")
    all_scores.append(rouge_l_fmeasure)




Example 1, ROUGE-L F-Score: 0.05128205128205128
Example 2, ROUGE-L F-Score: 0.10344827586206896
Example 3, ROUGE-L F-Score: 0.07999999999999999
Example 4, ROUGE-L F-Score: 0.18181818181818182
Example 5, ROUGE-L F-Score: 0.09523809523809525
Example 6, ROUGE-L F-Score: 0.0816326530612245
Example 7, ROUGE-L F-Score: 0.196078431372549
Example 8, ROUGE-L F-Score: 0.11111111111111112
Example 9, ROUGE-L F-Score: 0.3870967741935483
Example 10, ROUGE-L F-Score: 0.06666666666666667
Example 11, ROUGE-L F-Score: 0.1408450704225352
Example 12, ROUGE-L F-Score: 0.15000000000000002
Example 13, ROUGE-L F-Score: 0.2641509433962264
Example 14, ROUGE-L F-Score: 0.0975609756097561
Example 15, ROUGE-L F-Score: 0.12765957446808512
Example 16, ROUGE-L F-Score: 0.05714285714285715
Example 17, ROUGE-L F-Score: 0.0816326530612245
Example 18, ROUGE-L F-Score: 0.19354838709677416
Example 19, ROUGE-L F-Score: 0.16326530612244897
Example 20, ROUGE-L F-Score: 0.13953488372093023
Example 21, ROUGE-L F-Score: 0.136363

In [5]:
print(f"Average ROUGE-L F-Score: {sum(all_scores) / len(all_scores)}")

Average ROUGE-L F-Score: 0.1523797177076681
